In [2]:
import calendar
import datetime
from datetime import timedelta ,datetime
startDate = datetime.strptime("20220723", "%Y%m%d")

date = startDate + timedelta(days=1)
year= date.year
month = date.month
day = date.day

#系统默认：星期一作为一周的第一天（即：0），星期日作为一周的最后一天（即：6）
print(calendar.weekday(year,month,day))
currentday =calendar.weekday(year,month,day)


6


In [6]:
import akshare as ak
from datetime import date,datetime
print(date.today().strftime("%Y%m%d"))

20220724


In [12]:
import akshare as ak
import calendar
stock_sse_deal_daily_df = ak.stock_sse_deal_daily(date="20200707")
print(stock_sse_deal_daily_df)
currentday = calendar.weekday(2020, 7, 7)
print(currentday)

     单日情况             股票            主板A         主板B           科创板     股票回购
0    市价总值  417343.040000  394116.130000  740.330000  22486.590000      NaN
1   平均市盈率      15.590000      14.960000    8.260000     97.760000  0.00000
2    成交笔数    3534.305200    3413.136900    3.808500    117.359300  0.00050
3     成交量     658.900000     647.320000    0.910000     10.330000  0.34000
4    成交金额    7951.680262    7387.377096    4.537814    557.263461  2.50189
5     挂牌数    1689.000000    1520.000000   50.000000    119.000000      NaN
6     换手率       1.904700       1.874400    0.612900      2.478200  0.00000
7  次新股换手率       2.110500       1.735200         NaN      2.431200      NaN
8    流通市值  342628.977340  338262.701464  740.328484   3625.947393      NaN
9   流通换手率       2.320100       2.183900    0.612900     15.368800  0.00000
1


In [7]:
import sys
sys.path.append(".")
import utils.mongodbUtil as mongodbUtil
from  datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
gdpCol = mongodbUtil.getCol("stock","gdp")
dailyCol = mongodbUtil.getCol("stock","daily_deal")


def  getGdpDate(year, quarter):
    if quarter == 0:
        return str(year-1)+"-12-01"
    if quarter ==4:
        return str(year)+'-12-01'
    return str(year)+'-0'+ str(quarter*3)+'-01'



gdpCursor = gdpCol.find({"date":{"$gt":"2018-03-01"}}).sort("date",1)
gdpDict={}
for gdpData in gdpCursor:
    gdpDict[gdpData["date"]]=gdpData


pipeline = [
    {'$match': {"type": {"$in": ["sh", "sz"]}}},
    {'$group': {'_id': "$date", "count":{"$sum":1},'market_amount': {'$sum': "$market_amount"},'deal_amount': {'$sum': "$deal_amount"}}},
    {"$sort": {"_id": 1}}
]
date=[]
amountGdpRate=[]
dailyCursor = dailyCol.aggregate(pipeline)
for dailyData in dailyCursor:
    if dailyData['count'] != 2:
        continue
    date.append(dailyData["_id"])
    dt = datetime.strptime(dailyData["_id"], "%Y%m%d") 
    quarter = int(dt.month/3)
    gdpDate = getGdpDate(dt.year,quarter)
    formatDate = datetime.strptime(gdpDate, "%Y-%m-%d")
    formatDate1 = (formatDate + relativedelta(months=-3) ).strftime("%Y-%m-%d") 
    formatDate2 = (formatDate + relativedelta(months=-6) ).strftime("%Y-%m-%d") 
    formatDate3 = (formatDate + relativedelta(months=-9) ).strftime("%Y-%m-%d") 
    gdp = gdpDict[gdpDate]['curGdp'] + gdpDict[formatDate1]['curGdp'] + gdpDict[formatDate2]['curGdp'] + gdpDict[formatDate3]['curGdp']
    amountGdpRate.append(round(dailyData['market_amount']/gdp,3))

import pyecharts.options as opts
from pyecharts.charts import Line

(
    Line()
    # 全局配置
    .set_global_opts(xaxis_opts=opts.AxisOpts(is_scale=True),yaxis_opts=opts.AxisOpts(is_scale=True),datazoom_opts=[opts.DataZoomOpts(type_="slider")])
    # x轴
    .add_xaxis(xaxis_data=date)
    # y轴
    .add_yaxis(
        series_name="",         # 数据所属集合的名称
        y_axis=amountGdpRate,            # y轴数据
        is_smooth=True,
      markpoint_opts=opts.MarkPointOpts(
            # 标记点数据
            data=[
                # MarkPointItem：标记点数据项
                opts.MarkPointItem(
                     # 标注名称
                    name="自定义标记点", 
                    
                    # 特殊的标注类型，用于标注最大值最小值等。可选:
                    # 'min' 最大值、'max' 最大值 、'average' 平均值。
                    # 自己试了一下，如果同时设置type_和coord，只会显示coord的标记点
                    # type_ = 'min',
                    type_ = 'min',
                    
                    
                    # 在使用 type 时有效，用于指定在哪个维度上指定最大值最小值，可以是 
                    # 0（xAxis, radiusAxis），
                    # 1（yAxis, angleAxis），默认使用第一个数值轴所在的维度。
                    value_index = None,
                    
                     # 在使用 type 时有效，用于指定在哪个维度上指定最大值最小值。这可以是维度的直接名称Optional[str]，
                    # 例如折线图时可以是 x、angle 等、candlestick 图时可以是 open、close 等维度名称。
                    value_dim = None,
                    
                    # 标注的坐标。坐标格式视系列的坐标系而定，可以是直角坐标系上的 x, y，
                    # 也可以是极坐标系上的 radius, angle。例如 [121, 2323]、['aa', 998]。
                    # coord=[date[2], amountGdpRate[2]], #这里是直角坐标系x轴第三个，y轴第三个
                    
                    # 标注值，可以不设。
                    # value=amountGdpRate[2],
                    
                    # 相对容器的屏幕 x 坐标，单位像素 Optional[Numeric]  就是标记点与屏幕x方向的距离（一般默认None就好）。
                    x = None,  #一般默认就好
                    
                    # 相对容器的屏幕 y 坐标，单位像素 Optional[Numeric]  就是标记点与屏幕y方向的距离（一般默认None就好）。
                    y = None,  #一般默认就好
                    
                    # 标记的图形
                    # ECharts 提供的标记类型包括 'circle', 'rect', 'roundRect', 'triangle', 
                    # 'diamond', 'pin', 'arrow', 'none'
                    # 可以通过 'image://url' 设置为图片，其中 URL 为图片的链接，或者 dataURI。
                    symbol = None,  #一般默认就好
                    
                    # 标记的大小，可以设置成诸如 10 这样单一的数字，也可以用数组分开表示宽和高，
                    # 例如 [20, 10] 表示标记宽为 20，高为 10。
                    symbol_size = None,  #一般默认就好
                    
                    # 标记点样式配置项，参考 `series_options.ItemStyleOpts`
                    itemstyle_opts = None,
                    ),
                    
                       opts.MarkPointItem(
                     # 标注名称
                    name="自定义标记点", 
                    
                    # 特殊的标注类型，用于标注最大值最小值等。可选:
                    # 'min' 最大值、'max' 最大值 、'average' 平均值。
                    # 自己试了一下，如果同时设置type_和coord，只会显示coord的标记点
                    # type_ = 'min',
                    type_ = 'max',
                    
                    
                    # 在使用 type 时有效，用于指定在哪个维度上指定最大值最小值，可以是 
                    # 0（xAxis, radiusAxis），
                    # 1（yAxis, angleAxis），默认使用第一个数值轴所在的维度。
                    value_index = None,
                    
                     # 在使用 type 时有效，用于指定在哪个维度上指定最大值最小值。这可以是维度的直接名称Optional[str]，
                    # 例如折线图时可以是 x、angle 等、candlestick 图时可以是 open、close 等维度名称。
                    value_dim = None,
                    
                    # 标注的坐标。坐标格式视系列的坐标系而定，可以是直角坐标系上的 x, y，
                    # 也可以是极坐标系上的 radius, angle。例如 [121, 2323]、['aa', 998]。
                    # coord=[date[2], amountGdpRate[2]], #这里是直角坐标系x轴第三个，y轴第三个
                    
                    # 标注值，可以不设。
                    # value=amountGdpRate[2],
                    
                    # 相对容器的屏幕 x 坐标，单位像素 Optional[Numeric]  就是标记点与屏幕x方向的距离（一般默认None就好）。
                    x = None,  #一般默认就好
                    
                    # 相对容器的屏幕 y 坐标，单位像素 Optional[Numeric]  就是标记点与屏幕y方向的距离（一般默认None就好）。
                    y = None,  #一般默认就好
                    
                    # 标记的图形
                    # ECharts 提供的标记类型包括 'circle', 'rect', 'roundRect', 'triangle', 
                    # 'diamond', 'pin', 'arrow', 'none'
                    # 可以通过 'image://url' 设置为图片，其中 URL 为图片的链接，或者 dataURI。
                    symbol = None,  #一般默认就好
                    
                    # 标记的大小，可以设置成诸如 10 这样单一的数字，也可以用数组分开表示宽和高，
                    # 例如 [20, 10] 表示标记宽为 20，高为 10。
                    symbol_size = None,  #一般默认就好
                    
                    # 标记点样式配置项，参考 `series_options.ItemStyleOpts`
                    itemstyle_opts = None,
                    )
                    
                    
                    ],
            
            
            # 也可以在这里设置 标记的图形。
            # ECharts 提供的标记类型包括 'circle', 'rect', 'roundRect', 'triangle', 
            # 'diamond', 'pin', 'arrow', 'none'
            # 可以通过 'image://url' 设置为图片，其中 URL 为图片的链接，或者 dataURI。
            symbol = None,  #一般默认就好
            
            # 以及设置标记的大小，
            symbol_size = None,  #一般默认就好
            
            # 标签配置项，参考 `series_options.LabelOpts`
            label_opts = opts.LabelOpts(position="inside", color="#fff"),          
        ),
    )
    # 输出为HTML文件
    .render_notebook()
)